In [2]:
import zarr
import os
import numpy as np
from tqdm.auto import tqdm
import cv2  
from PIL import Image
from skimage.util import img_as_ubyte
from skimage.io import imsave
import napari

In [3]:
os.getcwd()

'/mnt/ka_gh_storage/Hi-DFA/Hi-DFA pipeline/Steps 1-2, 4-6'

In [4]:
trenches_zarr = zarr.open("trenches.zarr", mode='r')
trenches_zarr.shape

(60019, 481, 1, 135, 24)

In [5]:
num_trenches = trenches_zarr.shape[0]
num_frames = trenches_zarr.shape[1]
y_shape = trenches_zarr.shape[3]
x_shape = trenches_zarr.shape[4]

# Get training data for re-training

In [28]:
empty_zarr = zarr.open("empty.zarr", 
                       mode="w", 
                       shape=(num_trenches*num_frames, y_shape, x_shape),
                       chunks=(num_frames, y_shape, x_shape),
                       dtype='uint8')

In [79]:
mother_id = 316
start_frame = mother_id * num_frames
end_frame = (mother_id * num_frames) + num_frames - 1
print(f"Frames {start_frame} to {end_frame}")

Frames 72364 to 72592


In [80]:
concat_zarr_split = zarr.open("concat_split.zarr", mode='r+')
# masks_split_filtered = zarr.open("masks_split_filtered.zarr", mode='r+')
masks_split_filtered = zarr.open("masks_split.zarr", mode='r+')
empty_zarr = zarr.open("empty.zarr", mode='r+')

import napari # Visualise filtered masks
viewer = napari.Viewer()
viewer.add_image(concat_zarr_split, name='trenches')
viewer.add_labels(masks_split_filtered, name = 'masks')
viewer.add_labels(empty_zarr, name = 'empty')
viewer.dims.set_point(0, start_frame)

In [71]:
masks_split_filtered.shape

(17000044, 115, 16)

In [32]:
# mark_all = True
# if mark_all:
#     empty_zarr[start_frame:end_frame+1, :10, :] = 1

In [81]:
# concat mother_ids of interest to make it easier to remove artefacts from all timepoints
mother_ids = [32, 232, 268, 316, 407, 501, 689, 755, 798, 888, 928, 48320, 73937, 74235]

# Suppose you know the number of frames per trench
num_frames_per_trench = num_frames  # Replace with your actual value

# Function to map trench number to frame indices
def get_frame_indices(trench_number, num_frames_per_trench):
    start_index = trench_number * num_frames_per_trench
    end_index = start_index + num_frames_per_trench
    return start_index, end_index

# Prepare the shape and dtype for the new Zarr arrays
num_mothers = len(mother_ids)
frame_height = concat_zarr_split.shape[1]  # Assuming shape is (total_frames, height, width)
frame_width = concat_zarr_split.shape[2] * num_frames_per_trench  # Width after concatenation
data_dtype = concat_zarr_split.dtype
mask_dtype = masks_split_filtered.dtype

# Create new Zarr arrays on disk
concat_frames_array = zarr.open('concat_frames.zarr', mode='w', shape=(num_mothers, frame_height, frame_width), dtype=data_dtype, chunks=(1, frame_height, frame_width))
concat_masks_array = zarr.open('concat_masks.zarr', mode='w', shape=(num_mothers, frame_height, frame_width), dtype=mask_dtype, chunks=(1, frame_height, frame_width))

for idx, mother_id in enumerate(mother_ids):
    # Get frame indices for this trench
    start_idx, end_idx = get_frame_indices(mother_id, num_frames_per_trench)
    
    # Extract frames
    frames = concat_zarr_split[start_idx:end_idx, :, :]  # Shape: (num_frames, height, width)
    masks = masks_split_filtered[start_idx:end_idx, :, :]
    
    # Initialize lists to hold adjusted masks and frames
    adjusted_masks = []
    adjusted_frames = []
    
    # Keep track of the maximum label value to offset labels in each frame
    max_label = 0
    
    for frame_idx in range(num_frames_per_trench):
        # Get the mask for the current frame
        mask = masks[frame_idx]
        
        # Adjust the mask labels to ensure uniqueness
        # Skip background (assuming background label is 0)
        mask_adjusted = np.where(mask > 0, mask + max_label, mask)
        
        # Update max_label for the next frame
        max_label = mask_adjusted.max()
        
        # Append the adjusted mask and frame to the lists
        adjusted_masks.append(mask_adjusted)
        adjusted_frames.append(frames[frame_idx])
    
    # Concatenate frames and masks side-by-side along the width axis
    frames_concat = np.concatenate(adjusted_frames, axis=1)  # Shape: (height, width * num_frames)
    masks_concat = np.concatenate(adjusted_masks, axis=1)
    
    # Save the concatenated frames and masks to the new Zarr arrays
    concat_frames_array[idx, :, :] = frames_concat
    concat_masks_array[idx, :, :] = masks_concat

In [83]:
concat_frames_array = zarr.open("concat_frames.zarr", mode='r+')
concat_masks_array = zarr.open("concat_masks.zarr", mode='r+')

import napari # Visualise filtered masks
viewer = napari.Viewer()
viewer.add_image(concat_frames_array, name='trenches')
viewer.add_labels(concat_masks_array, name = 'masks')

<Labels layer 'masks' at 0x7ec13d35be90>

In [87]:
# Number of concatenated images
num_images = concat_frames_array.shape[0]

for i in range(num_images):
    frame = concat_frames_array[i]
    frame_max = frame.max()
    if frame_max > 0:
        frame = frame / frame_max
    else:
        frame = frame  # If max is zero, leave frame as is (all zeros)

    frame = img_as_ubyte(frame)

    # Save the frame
    index_str = str(i).zfill(3)
    im = Image.fromarray(frame)
    im.save(f"/home/ka537/external_SSD/Real Training Data/10_11_24/10_11_24_1_backportChip_wholeTrenchCrop_1_{index_str}.png")

    # Process mask
    mask = concat_masks_array[i]
    mask_max = mask.max()

    if mask_max <= 255:
        mask = mask.astype(np.uint8)
        im2 = Image.fromarray(mask)
    elif mask_max <= 65535:
        mask = mask.astype(np.uint16)
        im2 = Image.fromarray(mask, mode='I;16')
    else:
        print(f"Warning: Mask at index {i} has labels exceeding 65535. Cannot save as PNG.")
        continue  # Skip saving this mask or handle accordingly

    # Save the mask
    im2.save(f"/home/ka537/external_SSD/Real Training Data/10_11_24/10_11_24_1_backportChip_wholeTrenchCrop_1_{index_str}_masks.png")


In [ ]:
#

In [13]:
x = 0

for i, array in enumerate(empty_zarr):
    if np.sum(array) > 0:
        x = x+1

x

39

In [62]:
from PIL import Image
from skimage.util import img_as_ubyte

concat_size = 10 #omnipose documentation says to aim for (512,512)
concat_frames = []
concat_idx = 0

for i, array in enumerate(empty_zarr):
    if np.sum(array) > 0:
        concat_frames.append(i)

        if len(concat_frames) == concat_size:
            concatenated_frame = np.concatenate([concat_zarr_split[frame_idx] for frame_idx in concat_frames], axis=1)
            concatenated_masks = np.concatenate([masks_split_filtered[frame_idx] for frame_idx in concat_frames], axis=1)

            # Save the concatenated frame
            index_str = str(concat_idx).zfill(3)
            
            concatenated_frame = concatenated_frame/concatenated_frame.max()
            concatenated_frame = img_as_ubyte(concatenated_frame)
            im = Image.fromarray(concatenated_frame)
            im.save(f"/home/ka537/internal_SSD/Real Training Data/30_09_24/30_09_24_4_v4.1chip_tallCrop{index_str}.png")
            
            im2 = Image.fromarray(concatenated_masks.astype(np.ubyte))
            im2.save(f"/home/ka537/internal_SSD/Real Training Data/30_09_24/30_09_24_4_v4.1chip_tallCrop{index_str}_masks.png")
            # Reset batch_frames for the next batch
            concat_frames = []
            concat_idx += 1

In [ ]:
### Re-train from images before splitting (i.e. all timepoints)

In [10]:
masks = zarr.open("masks.zarr", mode='r')
empty_zarr = zarr.open("empty.zarr", 
                       mode="w", 
                       shape=masks.shape,
                       chunks=masks.chunks,
                       dtype='uint8')

In [11]:
concat = zarr.open("concat.zarr", mode='r+')
masks = zarr.open("masks.zarr", mode='r+')
empty_zarr = zarr.open("empty.zarr", mode='r+')

In [18]:
import napari # Visualise filtered masks
viewer = napari.Viewer()
viewer.add_image(concat, name='trenches')
viewer.add_labels(masks, name = 'masks')
viewer.add_labels(empty_zarr, name = 'empty')
viewer.dims.set_point(0, 6305)

In [132]:
# # Quickly delete all makss from a frame
# masks[frame] = np.zeros_like(masks[frame])

In [19]:
x = 0
frames = []

for i, array in enumerate(empty_zarr):
    if np.sum(array) > 0:
        x = x+1
        frames.append(i)

print(f"{x} images")
print(frames)

15 images
[1, 8, 6286, 6304, 6305, 9495, 9496, 9521, 34551, 34552, 58327, 61482, 61516, 67850, 67904]


In [20]:
for i, array in enumerate(empty_zarr):
    # Check if the sum of the current array in empty_zarr is greater than 0
    if np.sum(array) > 0:
        # Normalize and convert the frame to ubyte
        frame = concat[i]
        frame = frame / frame.max()
        frame = img_as_ubyte(frame)

        # Save the frame
        index_str = str(i).zfill(3)
        im = Image.fromarray(frame)
        im.save(f"/home/ka537/external_SSD/Real Training Data/16_11_24/16_11_24_1_backportChip_wholeTrenchCrop{index_str}.png")

        # Save the corresponding mask
        mask = masks[i].astype(np.ubyte)
        im2 = Image.fromarray(mask)
        im2.save(f"/home/ka537/external_SSD/Real Training Data/16_11_24/16_11_24_1_backportChip_wholeTrenchCrop{index_str}_masks.png")

In [84]:
short_FOVs = [111, 6380, 6390, 6401, 20701, 20703, 20707, 20823]
long_FOVs = [fov for fov in frames if fov not in short_FOVs]

In [123]:
# Mix images togther before saving (seperately for short vs long)
def process_and_shuffle_frames(fov_list, num_images_with_data, num_images_total, output_prefix):
    # Collect all individual images and masks across all frames
    all_images = []
    all_masks = []
    
    total_width = None  # Will be set later
    total_height = None  # Will be set later

    print(f"Collecting images from {len(fov_list)} frames for {output_prefix}")

    for fov in fov_list:
        # Directly access the frame and mask in `concat` using `fov` as index
        frame = concat[fov]
        mask = masks[fov]
        
        # Track non-blank images per FOV
        fov_non_blank_count = 0

        # Compute per-image width
        if total_width is None:
            total_width = frame.shape[1]
            total_height = frame.shape[0]
            per_image_width = total_width // num_images_total
            if total_width % num_images_total != 0:
                raise ValueError(f"Frame width {total_width} is not divisible by num_images_total {num_images_total}")

        # Calculate the width of the data-containing portion
        width_with_data = per_image_width * num_images_with_data

        # Extract the data portion
        frame_data = frame[:, :width_with_data]
        mask_data = mask[:, :width_with_data]

        # Deconcatenate the images and masks
        individual_images = np.hsplit(frame_data, num_images_with_data)
        individual_masks = np.hsplit(mask_data, num_images_with_data)

        # Filter out blank images
        for img, msk in zip(individual_images, individual_masks):
            if np.max(img) > 0:
                all_images.append(img)
                all_masks.append(msk)
                fov_non_blank_count += 1

        print(f"FOV {fov} contributed {fov_non_blank_count} non-blank images")

    total_images = len(all_images)
    print(f"Total non-blank images collected: {total_images}")

    # Verify that we have images to process
    if not all_images:
        print(f"No non-blank images found in {output_prefix}")
        return

    # Shuffle images and masks together
    combined = list(zip(all_images, all_masks))
    np.random.shuffle(combined)
    shuffled_images, shuffled_masks = zip(*combined)

    # Reassemble images and masks into frames
    images_per_frame = num_images_with_data  # Number of images per frame

    new_frames = []
    new_masks = []

    num_frames = math.ceil(total_images / images_per_frame)

    for i in range(num_frames):
        start_idx = i * images_per_frame
        end_idx = min(start_idx + images_per_frame, total_images)
        images_batch = list(shuffled_images[start_idx:end_idx])
        masks_batch = list(shuffled_masks[start_idx:end_idx])

        # Handle frames with fewer images (last frame)
        if len(images_batch) < images_per_frame:
            num_missing = images_per_frame - len(images_batch)
            zero_image = np.zeros((total_height, per_image_width), dtype=all_images[0].dtype)
            zero_mask = np.zeros((total_height, per_image_width), dtype=all_masks[0].dtype)
            images_batch.extend([zero_image] * num_missing)
            masks_batch.extend([zero_mask] * num_missing)

        # Concatenate images and masks horizontally
        frame_data = np.hstack(images_batch)
        mask_data = np.hstack(masks_batch)

        # Add the buffer back if necessary
        padding_width = total_width - (per_image_width * num_images_with_data)
        if padding_width > 0:
            frame_padding = np.zeros((total_height, padding_width), dtype=frame_data.dtype)
            mask_padding = np.zeros((total_height, padding_width), dtype=mask_data.dtype)
            frame_data = np.hstack((frame_data, frame_padding))
            mask_data = np.hstack((mask_data, mask_padding))

        new_frames.append(frame_data)
        new_masks.append(mask_data)

    # Save the reassembled frames and masks
    output_dir = "/home/ka537/external_SSD/Real Training Data/28_10_24/"  # Replace with your actual directory
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for idx, (frame_data, mask_data) in enumerate(zip(new_frames, new_masks)):
        # Normalize and convert the frame to ubyte
        if frame_data.max() > 0:
            frame_normalized = frame_data / frame_data.max()
        else:
            frame_normalized = frame_data  # Avoid division by zero
        frame_ubyte = img_as_ubyte(frame_normalized)

        # Convert mask to ubyte
        mask_ubyte = mask_data.astype(np.ubyte)

        # Save the frame
        index_str = f"{output_prefix}_{str(idx).zfill(5)}"
        frame_filename = os.path.join(output_dir, f"28_10_24_1_backportChip_wholeTrenchCrop_{index_str}.png")
        mask_filename = os.path.join(output_dir, f"28_10_24_1_backportChip_wholeTrenchCrop_{index_str}_masks.png")

        print(f"Saving frame {idx+1}/{len(new_frames)} as {frame_filename}")

        try:
            im_frame = Image.fromarray(frame_ubyte)
            im_frame.save(frame_filename)
        except Exception as e:
            print(f"Error saving frame {index_str}: {e}")

        # Save the corresponding mask
        try:
            im_mask = Image.fromarray(mask_ubyte)
            im_mask.save(mask_filename)
        except Exception as e:
            print(f"Error saving mask {index_str}_masks: {e}")

# Process and shuffle short_FOVs
process_and_shuffle_frames(
    fov_list=short_FOVs,
    num_images_with_data=50,     # Number of images with data in each frame
    num_images_total=237,        # Total images side-by-side including buffer
    output_prefix="short_FOV"
)

# Process and shuffle long_FOVs
process_and_shuffle_frames(
    fov_list=long_FOVs,
    num_images_with_data=237,    # All images contain data
    num_images_total=237,        # Total images side-by-side
    output_prefix="long_FOV"
)

FOV 111 contributed 50 non-blank images
FOV 6380 contributed 50 non-blank images
FOV 6390 contributed 50 non-blank images
FOV 6401 contributed 50 non-blank images
FOV 20701 contributed 50 non-blank images
FOV 20703 contributed 50 non-blank images
FOV 20707 contributed 50 non-blank images
FOV 20823 contributed 50 non-blank images
Total non-blank images collected: 400
Saving frame 1/8 as /home/ka537/external_SSD/Real Training Data/28_10_24/28_10_24_1_backportChip_wholeTrenchCrop_short_FOV_00000.png
Saving frame 2/8 as /home/ka537/external_SSD/Real Training Data/28_10_24/28_10_24_1_backportChip_wholeTrenchCrop_short_FOV_00001.png
Saving frame 3/8 as /home/ka537/external_SSD/Real Training Data/28_10_24/28_10_24_1_backportChip_wholeTrenchCrop_short_FOV_00002.png
Saving frame 4/8 as /home/ka537/external_SSD/Real Training Data/28_10_24/28_10_24_1_backportChip_wholeTrenchCrop_short_FOV_00003.png
Saving frame 5/8 as /home/ka537/external_SSD/Real Training Data/28_10_24/28_10_24_1_backportChip_wh

In [117]:
# Access and inspect the raw pixel values of frame 20823
# frame_index = frame_to_index[111]
frame_20823 = concat[20823]

# Print some basic statistics about the pixel values
print("Frame 20823 - Raw data statistics:")
print(f"Max pixel value: {frame_20823.max()}")
print(f"Min pixel value: {frame_20823.min()}")
print(f"Mean pixel value: {frame_20823.mean()}")

Frame 20823 - Raw data statistics:
Max pixel value: 65535
Min pixel value: 0
Mean pixel value: 2086.744638566573


In [119]:
# Accessing frame 20823 using the updated frame_to_index
frame_20823 = concat[frame_to_index[20823]]
print("Frame 20823 successfully accessed with frame_to_index:")
print(f"Max pixel value: {frame_20823.max()}")
print(f"Mean pixel value: {frame_20823.mean()}")

Frame 20823 successfully accessed with frame_to_index:
Max pixel value: 0
Mean pixel value: 0.0


In [36]:
# 2nd version
# Split first few frames of a trench with cells at all timepoints and add timepoint(s) to the start of all other trenches (so we can use empty trenches for training)
from PIL import Image
from skimage.util import img_as_ubyte
import numpy as np

first_two_FOVs_found = 0
first_FOV_frames = []
first_FOV_masks = []

# Set the number of frames to concatenate from the first FOV
concat_num_frames = 12  # Change this value as needed
num_images_concatenated = trenches_zarr.shape[1]

for i, array in enumerate(empty_zarr):
    if np.sum(array) > 0:
        if first_two_FOVs_found < 2:
            first_two_FOVs_found += 1
            # Deconcatenate the first two FOVs
            frame = concat[i]
            mask = masks[i]

            # Calculate the width of a single image
            image_width = frame.shape[1] // num_images_concatenated

            # Split the concatenated images into individual frames and masks
            for j in range(num_images_concatenated):
                start_col = j * image_width
                end_col = (j + 1) * image_width
                single_frame = frame[:, start_col:end_col]
                single_mask = mask[:, start_col:end_col]
                first_FOV_frames.append(single_frame)
                first_FOV_masks.append(single_mask)
        else:
            # For subsequent FOVs, concatenate 'concat_num_frames' images from the first two FOVs
            frame = concat[i]
            mask = masks[i]

            if len(first_FOV_frames) >= concat_num_frames:
                # Get 'concat_num_frames' frames and masks from the deconcatenated list
                left_frames = [first_FOV_frames.pop(0) for _ in range(concat_num_frames)]
                left_masks = [first_FOV_masks.pop(0) for _ in range(concat_num_frames)]

                # Concatenate the selected frames and masks horizontally
                left_frame_concat = np.concatenate(left_frames, axis=1)
                left_mask_concat = np.concatenate(left_masks, axis=1)

                # Concatenate with the current frame and mask
                frame = np.concatenate((left_frame_concat, frame), axis=1)
                mask = np.concatenate((left_mask_concat, mask), axis=1)
            else:
                # If not enough frames left, concatenate all remaining frames
                if len(first_FOV_frames) > 0:
                    left_frame_concat = np.concatenate(first_FOV_frames, axis=1)
                    left_mask_concat = np.concatenate(first_FOV_masks, axis=1)

                    # Clear the lists as we've used all remaining frames
                    first_FOV_frames.clear()
                    first_FOV_masks.clear()

                    # Concatenate with the current frame and mask
                    frame = np.concatenate((left_frame_concat, frame), axis=1)
                    mask = np.concatenate((left_mask_concat, mask), axis=1)
                else:
                    # No frames left, proceed without concatenation
                    pass

            # Normalize and convert to ubyte
            frame_normalized = frame / frame.max()
            frame_ubyte = img_as_ubyte(frame_normalized)

            # Save the frame
            index_str = str(i).zfill(3)
            im = Image.fromarray(frame_ubyte)
            im.save(f"/home/ka537/external_SSD/Real Training Data/28_10_24/28_10_24_1_backportChip_wholeTrenchCrop{index_str}.png")

            # Save the corresponding mask
            mask_ubyte = mask.astype(np.ubyte)
            im2 = Image.fromarray(mask_ubyte)
            im2.save(f"/home/ka537/external_SSD/Real Training Data/28_10_24/28_10_24_1_backportChip_wholeTrenchCrop{index_str}_masks.png")
    else:
        continue  # Skip arrays where the sum is not greater than 0

# After processing, re-concatenate the remaining frames from the first two FOVs
if len(first_FOV_frames) > 0:
    remaining_frame = np.concatenate(first_FOV_frames, axis=1)
    remaining_mask = np.concatenate(first_FOV_masks, axis=1)

    # Normalize and convert to ubyte
    remaining_frame_normalized = remaining_frame / remaining_frame.max()
    remaining_frame_ubyte = img_as_ubyte(remaining_frame_normalized)

    # Save the remaining frame
    index_str = "remaining_first_two_FOVs"
    im = Image.fromarray(remaining_frame_ubyte)
    im.save(f"/home/ka537/external_SSD/Real Training Data/28_10_24/28_10_24_1_backportChip_wholeTrenchCrop_{index_str}.png")

    # Save the corresponding mask
    remaining_mask_ubyte = remaining_mask.astype(np.ubyte)
    im2 = Image.fromarray(remaining_mask_ubyte)
    im2.save(f"/home/ka537/external_SSD/Real Training Data/28_10_24/28_10_24_1_backportChip_wholeTrenchCrop_{index_str}_masks.png")

In [ ]:
# Deconcat and discard some previous training data

In [39]:
# Parameters
concat_size = 10  # Number of concatenated frames

# Define the directory where the concatenated images are stored
image_dir = "/home/ka537/internal_SSD/Real Training Data/30_09_24/"

# Calculate the number of concatenated images based on the number of files
concat_idx = len([f for f in os.listdir(image_dir) if f.endswith('.png') and not 'masks' in f])

# Load one concatenated image to retrieve the dimensions
sample_image = np.array(Image.open(f"{image_dir}30_09_24_v4.1chip_tallCrop000.png"))

# Get frame height and calculate the original frame width
frame_height = sample_image.shape[0]
frame_width = sample_image.shape[1] // concat_size  # Divide total width by the number of concatenated frames

# Define paths to the concatenated images
concat_paths = [f"{image_dir}30_09_24_v4.1chip_tallCrop{str(idx).zfill(3)}.png" for idx in range(concat_idx)]
mask_paths = [f"{image_dir}30_09_24_v4.1chip_tallCrop{str(idx).zfill(3)}_masks.png" for idx in range(concat_idx)]

# Prepare Zarr arrays for storing the split frames and masks
concat_temp = zarr.open('concat_temp.zarr', mode='w', shape=(len(concat_paths) * concat_size, frame_height, frame_width), dtype=np.uint8)
masks_temp = zarr.open('masks_temp.zarr', mode='w', shape=(len(mask_paths) * concat_size, frame_height, frame_width), dtype=np.uint8)
empty_temp = zarr.open('empty_temp.zarr', mode='w', shape=(len(mask_paths) * concat_size, frame_height, frame_width), dtype=np.uint8)

for idx, (concat_path, mask_path) in enumerate(zip(concat_paths, mask_paths)):
    # Load concatenated image and mask
    concatenated_image = np.array(Image.open(concat_path))
    concatenated_mask = np.array(Image.open(mask_path))

    # Split the concatenated image and mask back into original frames
    for i in range(concat_size):
        start_col = i * frame_width
        end_col = (i + 1) * frame_width

        split_frame = concatenated_image[:, start_col:end_col]
        split_mask = concatenated_mask[:, start_col:end_col]

        # Store split frames and masks into Zarr arrays
        concat_temp[idx * concat_size + i] = split_frame
        masks_temp[idx * concat_size + i] = split_mask

In [43]:
concat_temp = zarr.open("concat_temp.zarr", mode='r+')
masks_temp = zarr.open("masks_temp.zarr", mode='r+')
empty_temp = zarr.open("empty_temp.zarr", mode='r+')

import napari # Visualise filtered masks
viewer = napari.Viewer()
viewer.add_image(concat_temp, name='trenches')
viewer.add_labels(masks_temp, name = 'masks')
viewer.add_labels(empty_temp, name = 'empty')

<Labels layer 'empty' at 0x71128ab1b510>

In [44]:
x = 0

for i, array in enumerate(empty_temp):
    if np.sum(array) == 0:
        x = x+1

x

200

In [47]:
from PIL import Image
from skimage.util import img_as_ubyte

concat_size = 10
concat_frames = []
concat_idx = 0

for i, array in enumerate(empty_temp):
    if np.sum(array) == 0:
        concat_frames.append(i)

        if len(concat_frames) == concat_size:
            concatenated_frame = np.concatenate([concat_temp[frame_idx] for frame_idx in concat_frames], axis=1)
            concatenated_masks = np.concatenate([masks_temp[frame_idx] for frame_idx in concat_frames], axis=1)

            # Save the concatenated frame
            index_str = str(concat_idx).zfill(3)
            
            concatenated_frame = concatenated_frame/concatenated_frame.max()
            concatenated_frame = img_as_ubyte(concatenated_frame)
            im = Image.fromarray(concatenated_frame)
            im.save(f"/home/ka537/internal_SSD/Real Training Data/30_09_24/30_09_24_v4.1chip_tallCrop{index_str}.png")
            
            im2 = Image.fromarray(concatenated_masks.astype(np.ubyte))
            im2.save(f"/home/ka537/internal_SSD/Real Training Data/30_09_24/30_09_24_v4.1chip_tallCrop{index_str}_masks.png")
            # Reset batch_frames for the next batch
            concat_frames = []
            concat_idx += 1

In [3]:
## Downscale 60x training data for training 30x model

In [8]:
import os
import numpy as np
from skimage.io import imread, imsave
from skimage.transform import resize

# Define input and output directories
input_dir = "/home/ka537/external_SSD/Real Training Data/60x binned 2x2 (for 30x)/"
output_dir = os.path.join(input_dir, "30x")

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Process each file
for file_name in os.listdir(input_dir):
    if file_name.endswith('.png') and not file_name.startswith('.'):
        # Determine if the file is an image or a mask
        if file_name.endswith('_masks.png'):
            base_name = file_name[:-10]  # Remove '_masks.png'
            is_mask = True
        else:
            base_name = file_name[:-4]   # Remove '.png'
            is_mask = False

        # Full path to the input file
        file_path = os.path.join(input_dir, file_name)

        # Open the image
        try:
            img = imread(file_path)
        except IOError:
            print(f"Cannot open image file {file_path}")
            continue

        # Determine the new size
        new_shape = (img.shape[0] // 2, img.shape[1] // 2)

        if is_mask:
            # For masks, use nearest neighbor interpolation (order=0)
            img_resized = resize(
                img,
                new_shape,
                order=0,
                preserve_range=True,
                anti_aliasing=False
            ).astype(img.dtype)
            new_file_name = f"{base_name}_30x_masks.png"
        else:
            # For images, use anti-aliasing and bilinear interpolation (order=1)
            img_resized = resize(
                img,
                new_shape,
                order=1,
                preserve_range=True,
                anti_aliasing=True
            ).astype(np.uint8)
            new_file_name = f"{base_name}_30x.png"

        # Full path to the output file
        output_path = os.path.join(output_dir, new_file_name)

        # Save the resized image
        imsave(output_path, img_resized)

        print(f"Processed {file_name} -> {new_file_name}")

Processed 30_09_24_4_v4.1chip_tallCrop049_masks.png -> 30_09_24_4_v4.1chip_tallCrop049_30x_masks.png
Processed 14_10_24_4_backportChip_wholeTrenchCrop041_masks.png -> 14_10_24_4_backportChip_wholeTrenchCrop041_30x_masks.png
Processed 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00016.png -> 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00016_30x.png
Processed 30_09_24_v4.1chip_tallCrop006.png -> 30_09_24_v4.1chip_tallCrop006_30x.png
Processed 14_10_24_2_backportChip_wholeTrenchCrop000_masks.png -> 14_10_24_2_backportChip_wholeTrenchCrop000_30x_masks.png
Processed 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00026_masks.png -> 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00026_30x_masks.png
Processed 30_09_24_v4.1chip_tallCrop002_masks.png -> 30_09_24_v4.1chip_tallCrop002_30x_masks.png
Processed 30_09_24_4_v4.1chip_tallCrop001_masks.png -> 30_09_24_4_v4.1chip_tallCrop001_30x_masks.png
Processed 30_09_24_4_v4.1chip_tallCrop014.png -> 30_09_24_4_v4.1chip_tallCrop014_30x.png

/tmp/ipykernel_65731/2308783207.py:62: UserWarning: /home/ka537/external_SSD/Real Training Data/60x binned 2x2 (for 30x)/30x/30_09_24_v4.1chip_tallCrop002_30x_masks.png is a low contrast image
  imsave(output_path, img_resized)
/tmp/ipykernel_65731/2308783207.py:62: UserWarning: /home/ka537/external_SSD/Real Training Data/60x binned 2x2 (for 30x)/30x/30_09_24_4_v4.1chip_tallCrop028_30x_masks.png is a low contrast image
  imsave(output_path, img_resized)
/tmp/ipykernel_65731/2308783207.py:62: UserWarning: /home/ka537/external_SSD/Real Training Data/60x binned 2x2 (for 30x)/30x/30_09_24_v4.1chip_tallCrop016_30x_masks.png is a low contrast image
  imsave(output_path, img_resized)
/tmp/ipykernel_65731/2308783207.py:62: UserWarning: /home/ka537/external_SSD/Real Training Data/60x binned 2x2 (for 30x)/30x/28_10_24_1_backportChip_wholeTrenchCrop_short_FOV_00004_30x_masks.png is a low contrast image
  imsave(output_path, img_resized)
/tmp/ipykernel_65731/2308783207.py:62: UserWarning: /home/ka

Processed 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00028.png -> 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00028_30x.png
Processed 30_09_24_v4.1chip_tallCrop016.png -> 30_09_24_v4.1chip_tallCrop016_30x.png
Processed 30_09_24_3_v4.1chip_tallCrop039_masks.png -> 30_09_24_3_v4.1chip_tallCrop039_30x_masks.png
Processed 14_10_24_4_backportChip_wholeTrenchCrop252_masks.png -> 14_10_24_4_backportChip_wholeTrenchCrop252_30x_masks.png
Processed 30_09_24_4_v4.1chip_tallCrop026.png -> 30_09_24_4_v4.1chip_tallCrop026_30x.png
Processed 30_09_24_4_v4.1chip_tallCrop022_masks.png -> 30_09_24_4_v4.1chip_tallCrop022_30x_masks.png
Processed 30_09_24_3_v4.1chip_tallCrop035_masks.png -> 30_09_24_3_v4.1chip_tallCrop035_30x_masks.png
Processed 30_09_24_3_v4.1chip_tallCrop001.png -> 30_09_24_3_v4.1chip_tallCrop001_30x.png
Processed 30_09_24_v4.1chip_tallCrop013.png -> 30_09_24_v4.1chip_tallCrop013_30x.png
Processed 30_09_24_4_v4.1chip_tallCrop037_masks.png -> 30_09_24_4_v4.1chip_tallCrop037_30x_

/tmp/ipykernel_65731/2308783207.py:62: UserWarning: /home/ka537/external_SSD/Real Training Data/60x binned 2x2 (for 30x)/30x/24_08_24_v4.1chip_000_30x_masks.png is a low contrast image
  imsave(output_path, img_resized)


Processed 14_10_24_4_backportChip_wholeTrenchCrop053.png -> 14_10_24_4_backportChip_wholeTrenchCrop053_30x.png
Processed 30_09_24_4_v4.1chip_tallCrop002.png -> 30_09_24_4_v4.1chip_tallCrop002_30x.png
Processed 30_09_24_2_v4.1chip_tallCrop004.png -> 30_09_24_2_v4.1chip_tallCrop004_30x.png
Processed 30_09_24_3_v4.1chip_tallCrop020_masks.png -> 30_09_24_3_v4.1chip_tallCrop020_30x_masks.png
Processed 30_09_24_3_v4.1chip_tallCrop005_masks.png -> 30_09_24_3_v4.1chip_tallCrop005_30x_masks.png
Processed 28_10_24_1_backportChip_wholeTrenchCrop_short_FOV_00003.png -> 28_10_24_1_backportChip_wholeTrenchCrop_short_FOV_00003_30x.png
Processed 30_09_24_4_v4.1chip_tallCrop051.png -> 30_09_24_4_v4.1chip_tallCrop051_30x.png
Processed 14_10_24_4_backportChip_wholeTrenchCrop420_masks.png -> 14_10_24_4_backportChip_wholeTrenchCrop420_30x_masks.png
Processed 14_10_24_2_backportChip_wholeTrenchCrop005.png -> 14_10_24_2_backportChip_wholeTrenchCrop005_30x.png
Processed 30_09_24_4_v4.1chip_tallCrop019_masks.p

...

Processed 14_10_24_4_backportChip_wholeTrenchCrop420.png -> 14_10_24_4_backportChip_wholeTrenchCrop420_30x.png
Processed 30_09_24_2_v4.1chip_tallCrop011.png -> 30_09_24_2_v4.1chip_tallCrop011_30x.png
Processed 30_09_24_4_v4.1chip_tallCrop035.png -> 30_09_24_4_v4.1chip_tallCrop035_30x.png
Processed 14_10_24_2_backportChip_wholeTrenchCrop004.png -> 14_10_24_2_backportChip_wholeTrenchCrop004_30x.png
Processed 30_09_24_2_v4.1chip_tallCrop017_masks.png -> 30_09_24_2_v4.1chip_tallCrop017_30x_masks.png
Processed 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00027.png -> 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00027_30x.png
Processed 30_09_24_4_v4.1chip_tallCrop044.png -> 30_09_24_4_v4.1chip_tallCrop044_30x.png
Processed 30_09_24_3_v4.1chip_tallCrop024.png -> 30_09_24_3_v4.1chip_tallCrop024_30x.png
Processed 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00000.png -> 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00000_30x.png
Processed 30_09_24_3_v4.1chip_tallCrop011_masks.png

/tmp/ipykernel_65731/2308783207.py:62: UserWarning: /home/ka537/external_SSD/Real Training Data/60x binned 2x2 (for 30x)/30x/30_09_24_4_v4.1chip_tallCrop016_30x_masks.png is a low contrast image
  imsave(output_path, img_resized)
/tmp/ipykernel_65731/2308783207.py:62: UserWarning: /home/ka537/external_SSD/Real Training Data/60x binned 2x2 (for 30x)/30x/30_09_24_4_v4.1chip_tallCrop026_30x_masks.png is a low contrast image
  imsave(output_path, img_resized)


Processed 14_10_24_4_backportChip_wholeTrenchCrop000.png -> 14_10_24_4_backportChip_wholeTrenchCrop000_30x.png
Processed 30_09_24_4_v4.1chip_tallCrop034.png -> 30_09_24_4_v4.1chip_tallCrop034_30x.png
Processed 30_09_24_4_v4.1chip_tallCrop036_masks.png -> 30_09_24_4_v4.1chip_tallCrop036_30x_masks.png
Processed 30_09_24_4_v4.1chip_tallCrop000_masks.png -> 30_09_24_4_v4.1chip_tallCrop000_30x_masks.png
Processed 30_09_24_3_v4.1chip_tallCrop021.png -> 30_09_24_3_v4.1chip_tallCrop021_30x.png
Processed 14_10_24_4_backportChip_wholeTrenchCrop041.png -> 14_10_24_4_backportChip_wholeTrenchCrop041_30x.png
Processed 30_09_24_v4.1chip_tallCrop009.png -> 30_09_24_v4.1chip_tallCrop009_30x.png
Processed 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00025.png -> 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00025_30x.png
Processed 30_09_24_3_v4.1chip_tallCrop030.png -> 30_09_24_3_v4.1chip_tallCrop030_30x.png
Processed 30_09_24_4_v4.1chip_tallCrop020.png -> 30_09_24_4_v4.1chip_tallCrop020_30x.png


/tmp/ipykernel_65731/2308783207.py:62: UserWarning: /home/ka537/external_SSD/Real Training Data/60x binned 2x2 (for 30x)/30x/30_09_24_v4.1chip_tallCrop000_30x_masks.png is a low contrast image
  imsave(output_path, img_resized)
/tmp/ipykernel_65731/2308783207.py:62: UserWarning: /home/ka537/external_SSD/Real Training Data/60x binned 2x2 (for 30x)/30x/28_10_24_1_backportChip_wholeTrenchCrop_short_FOV_00001_30x_masks.png is a low contrast image
  imsave(output_path, img_resized)


Processed 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00024.png -> 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00024_30x.png
Processed 30_09_24_4_v4.1chip_tallCrop048.png -> 30_09_24_4_v4.1chip_tallCrop048_30x.png
Processed 30_09_24_4_v4.1chip_tallCrop034_masks.png -> 30_09_24_4_v4.1chip_tallCrop034_30x_masks.png
Processed 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00021.png -> 28_10_24_1_backportChip_wholeTrenchCrop_long_FOV_00021_30x.png
Processed 30_09_24_3_v4.1chip_tallCrop037_masks.png -> 30_09_24_3_v4.1chip_tallCrop037_30x_masks.png
Processed 30_09_24_3_v4.1chip_tallCrop025.png -> 30_09_24_3_v4.1chip_tallCrop025_30x.png
Processed 30_09_24_2_v4.1chip_tallCrop010.png -> 30_09_24_2_v4.1chip_tallCrop010_30x.png
Processed 30_09_24_v4.1chip_tallCrop003_masks.png -> 30_09_24_v4.1chip_tallCrop003_30x_masks.png
Processed 30_09_24_3_v4.1chip_tallCrop032.png -> 30_09_24_3_v4.1chip_tallCrop032_30x.png
Processed 30_09_24_3_v4.1chip_tallCrop001_masks.png -> 30_09_24_3_v4.1chip_

In [ ]:
###

In [ ]:
## TAKING 2x2 UPSCALED DATA FOR RETRAINING

In [4]:
import napari
viewer = napari.Viewer()
viewer.add_image(zarr.open("train_images.zarr", mode="r+"), name="Training Images")
viewer.add_labels(zarr.open("train_masks.zarr", mode="r+"), name="Training Masks")

<Labels layer 'Training Masks' at 0x7383d6207a50>

In [15]:
# export to PNG
from skimage.io import imsave
train_images = zarr.open("train_images.zarr", mode="r")
train_masks = zarr.open("train_masks.zarr", mode="r")

# Create output directories for images and masks.
os.makedirs("train_png", exist_ok=True)
os.makedirs("train_masks_png", exist_ok=True)

# Loop over the 50 examples and export each as a PNG.
for i in range(len(train_images)):
    img = train_images[i]
    mask = train_masks[i]
    # Save with a name that includes a zero-padded index.
    imsave(os.path.join("train_png", f"07_02_25_backportChip_40x_upscaled2x2_{i:03d}.png"), img)
    imsave(os.path.join("train_masks_png", f"07_02_25_backportChip_40x_upscaled2x2_{i:03d}_masks.png"), mask)

/tmp/ipykernel_173707/93994850.py:16: UserWarning: train_masks_png/06_02_25_backportChip_40x_upscaled2x2_000_masks.png is a low contrast image
  imsave(os.path.join("train_masks_png", f"06_02_25_backportChip_40x_upscaled2x2_{i:03d}_masks.png"), mask)
/tmp/ipykernel_173707/93994850.py:16: UserWarning: train_masks_png/06_02_25_backportChip_40x_upscaled2x2_001_masks.png is a low contrast image
  imsave(os.path.join("train_masks_png", f"06_02_25_backportChip_40x_upscaled2x2_{i:03d}_masks.png"), mask)
/tmp/ipykernel_173707/93994850.py:16: UserWarning: train_masks_png/06_02_25_backportChip_40x_upscaled2x2_002_masks.png is a low contrast image
  imsave(os.path.join("train_masks_png", f"06_02_25_backportChip_40x_upscaled2x2_{i:03d}_masks.png"), mask)
/tmp/ipykernel_173707/93994850.py:16: UserWarning: train_masks_png/06_02_25_backportChip_40x_upscaled2x2_003_masks.png is a low contrast image
  imsave(os.path.join("train_masks_png", f"06_02_25_backportChip_40x_upscaled2x2_{i:03d}_masks.png"), ma

## Version to manually define FOVs to export

In [7]:
masks = zarr.open("masks_upscaled_filtered.zarr", mode="r+")
include_channel = zarr.open("include_channel.zarr", mode="w",
                            shape=masks.shape, dtype='uint8')

In [13]:
#############################################
# 1. Open the Zarr arrays
#############################################
trenches = zarr.open("trenches.zarr", mode="r")
masks = zarr.open("masks_upscaled_filtered.zarr", mode="r+")
include_channel = zarr.open("include_channel.zarr", mode="r+")

#############################################
# 2. Prepare Napari viewer & marked list
#############################################
viewer = napari.Viewer()
viewer.add_image(trenches, name='trenches', scale=(1, 1, 1, 2, 2))
viewer.add_labels(masks, name='masks')
viewer.add_labels(include_channel, name='include_channel')

# <-- this list will collect your picks -->
marked_list = []

#############################################
# 3. Define the hotkey callback for 'l'
#############################################
def mark_current_frame(event):
    # get the current indices (fov, frame, channel)
    fov_idx, frame_idx, ch_idx = viewer.dims.current_step[:3]
    print(f"Marking FOV {fov_idx}, Frame {frame_idx}, Channel {ch_idx}")

    # label top 10% rows of include_channel
    height = include_channel.shape[3]
    top_rows = int(height * 0.1)
    include_channel[fov_idx, frame_idx, ch_idx, :top_rows, :] = 1

    # refresh the viewer layer
    viewer.layers['include_channel'].data = include_channel

    # record selection
    marked_list.append({'fov': int(fov_idx), 'frame': int(frame_idx)})

    # switch back to masks for convenience
    viewer.layers.selection.active = viewer.layers['masks']

# bind 'l'
viewer.bind_key('l', mark_current_frame)

#############################################
# 4. Run Napari
#############################################
napari.run()

Marking FOV 54302, Frame 34, Channel 0


In [14]:
print("Marked frames:", marked_list)

Makred frames: [{'fov': 54302, 'frame': 34}]


In [12]:
#############################################
# 1. Open the Zarr arrays
#############################################
trenches        = zarr.open("trenches.zarr", mode="r")
masks           = zarr.open("masks_upscaled_filtered.zarr", mode="r")
include_channel = zarr.open("include_channel.zarr", mode="r")

#############################################
# 2. Export settings
#############################################
group_size = 6
os.makedirs("train_png", exist_ok=True)
os.makedirs("train_masks_png", exist_ok=True)

#############################################
# 3. Build the list of (fov, frame) pairs
#    from marked_list (already in namespace)
#############################################
coords = [(int(item['fov']), int(item['frame'])) for item in marked_list]
coords.sort()

#############################################
# 4. Loop through coords and batch-export
#############################################
batch_trenches = []
batch_masks    = []
batch_idx      = 0

pbar = tqdm(total=len(coords), desc="Exporting marked frames")
for fov, t in coords:
    batch_trenches.append(trenches[fov, t, 0, :, :])
    batch_masks.append(masks[fov, t, 0, :, :])

    if len(batch_trenches) == group_size:
        # Upscale and concatenate trenches
        upscaled = [
            cv2.resize(
                frm,
                (frm.shape[1] * 2, frm.shape[0] * 2),
                interpolation=cv2.INTER_CUBIC
            )
            for frm in batch_trenches
        ]
        concat_tr = np.concatenate(upscaled, axis=1)
        concat_ms = np.concatenate(batch_masks, axis=1)

        # Cast and save
        concat_tr = img_as_ubyte(concat_tr)
        concat_ms = concat_ms.astype(np.uint8)
        Image.fromarray(concat_tr).save(f"train_png/group_{batch_idx:03d}_2.png")
        Image.fromarray(concat_ms).save(f"train_masks_png/group_{batch_idx:03d}_2_masks.png")

        batch_trenches.clear()
        batch_masks.clear()
        batch_idx += 1

    pbar.update(1)

pbar.close()

# Final partial batch, if any remain
if batch_trenches:
    upscaled = [
        cv2.resize(
            frm,
            (frm.shape[1] * 2, frm.shape[0] * 2),
            interpolation=cv2.INTER_CUBIC
        )
        for frm in batch_trenches
    ]
    concat_tr = np.concatenate(upscaled, axis=1)
    concat_ms = np.concatenate(batch_masks, axis=1)

    concat_tr = img_as_ubyte(concat_tr)
    concat_ms = concat_ms.astype(np.uint8)
    Image.fromarray(concat_tr).save(f"train_png/group_{batch_idx:03d}_2.png")
    Image.fromarray(concat_ms).save(f"train_masks_png/group_{batch_idx:03d}_2_masks.png")

Exporting marked frames:   0%|          | 0/83 [00:00<?, ?it/s]

In [ ]:
##

# cmd line for omnipose model training/retraining

In [ ]:
# Train from scratch
conda activate omnipose
CUDA_VISIBLE_DEVICES=0 python -m omnipose --train --use_gpu --dir '/home/ka537/internal_SSD/Real Training Data/' --mask_filter "_masks" --n_epochs 4000 --pretrained_model None --save_every 200 --save_each --learning_rate 0.1 --diameter 0 --batch_size 16 --RAdam --nclasses 2 --nchan 1 --look_one_level_down

In [ ]:
# Retrain model
conda activate omnipose
CUDA_VISIBLE_DEVICES=0 python -m omnipose --train --use_gpu --dir '/home/ka537/external_SSD/Real Training Data/16_11_24/' --mask_filter "_masks" --n_epochs 4000 --pretrained_model '/home/ka537/external_SSD/Real Training Data/60x binned 2x2 (for 30x)/30x/models/cellpose_residual_on_style_on_concatenation_off_omni_abstract_nclasses_2_nchan_1_dim_2_30x_2024_11_14_20_48_30.434906_epoch_3999' --save_every 200 --save_each --learning_rate 0.1 --diameter 0 --batch_size 16 --RAdam --nclasses 2 --nchan 1

In [ ]:
# Retrain model with tyx 128,128 (default is 224,224 -- should be about half of image size) ## THIS SEEMS WORSE THAN DEFAULT FOR 280x296 IMAGES!!
conda activate omnipose
CUDA_VISIBLE_DEVICES=0 python -m omnipose --train --use_gpu --dir '/home/ka537/external_SSD/Real Training Data/18_04_25 (40x, 6 frames concat, 2x2 upscaled)/' --mask_filter "_masks" --n_epochs 4000 --pretrained_model '/home/ka537/external_SSD/Real Training Data/22_01_25 (40x, 6 frames concat, 2x2 upscaled)/models (most recent without blurry images)/cellpose_residual_on_style_on_concatenation_off_omni_abstract_nclasses_2_nchan_1_dim_2_22_01_25 (40x, 6 frames concat, 2x2 upscaled)_2025_02_11_01_30_46.281520_epoch_3999' --save_every 200 --save_each --learning_rate 0.1 --diameter 0 --batch_size 16 --RAdam --nclasses 2 --nchan 1 --tyx 128,128

In [ ]:
#